In [1]:
import numpy as np
import pyvo

In [31]:
service = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
result = service.run_async("""
           SELECT ra,dec,j_m,j_msigcom,h_m,h_msigcom,k_m,k_msigcom,ph_qual,cc_flg
           FROM fp_psc
           WHERE CONTAINS(POINT('ICRS',ra, dec), CIRCLE('ICRS',202.48417,47.23056,0.4))=1
    """)

# Wait for 23s.

In [16]:
tab = result.to_table()
print(tab)

    ra        dec      j_m   j_msigcom ...  k_m   k_msigcom ph_qual cc_flg
   deg        deg      mag      mag    ...  mag      mag                  
---------- ---------- ------ --------- ... ------ --------- ------- ------
202.900750  46.961285 16.168     0.096 ... 15.180     0.158     ABC    000
202.951614  47.024986 15.773     0.072 ... 15.541     0.234     ABD    000
202.922589  47.024452 14.628     0.032 ... 14.036     0.059     AAA    000
202.911833  47.011093 13.948     0.025 ... 13.318     0.036     AAA    000
202.925932  47.004223 16.461     0.131 ... 17.007        --     BCU    000
202.515450  46.929302 15.967     0.088 ... 15.077     0.140     AAB    000
202.532240  46.931587 16.575     0.145 ... 15.888        --     BDU    000
202.607930  46.932255 16.658     0.147 ... 15.430     0.193     BUC    000
202.823902  47.011593 16.555     0.143 ... 16.136        --     BBU    000
202.809023  46.964558 15.874     0.081 ... 15.322     0.188     AAC    000
       ...        ...    

In [38]:
async_srv = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
job = async_srv.submit_job("""
            SELECT m.cntr, m.ra, m.dec, m.j_m, m.k_m, w.ra, w.dec
            FROM fp_psc AS m, allwise_p3as_psd AS w
            WHERE CONTAINS(POINT('ICRS', m.ra, m.dec), CIRCLE('ICRS',202.48417,47.23056,0.4))=1
            """)
job.run()

In [43]:
job.fetch_result()

<Table length=695>
   cntr        ra        dec       j_m     k_m  
              deg        deg       mag     mag  
  int64     float64    float64   float32 float32
---------- ---------- ---------- ------- -------
1312843203 202.900750  46.961285  16.168  15.180
1312846041 202.951614  47.024986  15.773  15.541
1312843291 202.922589  47.024452  14.628  14.036
1312843272 202.911833  47.011093  13.948  13.318
1312843265 202.925932  47.004223  16.461  17.007
1312825176 202.515450  46.929302  15.967  15.077
1312825178 202.532240  46.931587  16.575  15.888
1312825179 202.607930  46.932255  16.658  15.430
1312843274 202.823902  47.011593  16.555  16.136
       ...        ...        ...     ...     ...
1312791194 202.073558  47.473503  16.402  15.465
1312791168 202.057425  47.489326  15.682  15.233
1312791202 202.011931  47.468540  15.544  14.927
1312791392 201.971808  47.342934  16.579  14.849
1312791455 201.953797  47.295887  18.527  16.888
1312791367 201.950370  47.356930   9.751   9.468
 

In [10]:
async_srv = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
job = async_srv.run_async("""
            SELECT TOP 10000 m.pts_key, m.ra, m.dec, m.j_m, m.j_msigcom, m.h_m, m.h_msigcom, m.k_m,
            m.k_msigcom, m.ph_qual, m.cc_flg, w.ra, w.dec, w.w1mpro, w.w2mpro,
            w.w1snr, w.w2snr
            FROM allwise_p3as_psd AS w,
            fp_psc AS m
            WHERE m.k_m - w.w2mpro >= 1.8 - 0.848 * (m.j_m - m.k_m) AND --(comment: JKW2 color cut)
            m.j_m > 0 AND
            w.w1snr >= 5 AND  --(comment: W1 and W2 S/N ratio > 5)
            w.w2snr >= 5
            """)
#job.run()

DALServiceError: Cannot wait for job completion. Job is not active!

#### Possible end product of the code

```python
async_srv = pyvo.dal.TAPService('https://irsa.ipac.caltech.edu/TAP')
job = async_srv.run_async("""
            SELECT TOP 10000 m.pts_key, m.ra, m.dec, m.j_m, m.j_msigcom, m.h_m, m.h_msigcom, m.k_m,
            m.k_msigcom, m.ph_qual, m.cc_flg, w.ra, w.dec, w.w1mpro, w.w2mpro,
            w.w1snr, w.w2snr
            FROM allwise_p3as_psd AS w
            CROSS APPLY (SELECT * FROM dbo.fGetNearestObjEq(w.ra, w.dec, 0.066)) AS x
            INNER JOIN fp_psc AS m on m.pts_key = x.pts.key
            WHERE m.k_m - w.w2mpro >= 1.8 - 0.848 * (m.j_m - m.k_m) AND --(comment: JKW2 color cut)
            m.j_m > 0 AND
            w.w1snr >= 5 AND  --(comment: W1 and W2 S/N ratio > 5)
            w.w2snr >= 5
            """)
#job.run()
#print(job.phase)
#tab = job.to_table()

In [25]:
tap_services = pyvo.regsearch(servicetype='tap', keywords=['2MASS point source'], includeaux=True)
tap_services.to_table()['ivoid', 'short_name', 'res_title']


ivoid,short_name,res_title
object,object,object
ivo://archive.stsci.edu/hscv2tap,HSCv2 TAP,MAST STScI Hubble Source Catalog Version 2 (HSCv2) TAP service
ivo://cds.vizier/i/280b,I/280B,All-Sky Compiled Catalogue of 2.5 million stars
ivo://cds.vizier/i/319,I/319,XPM Catalog of positions and proper motions
ivo://cds.vizier/ii/241,II/241,2MASS Catalog Incremental Data Release
ivo://cds.vizier/ii/246,II/246,2MASS All-Sky Catalog of Point Sources
ivo://cds.vizier/ii/270,II/270,TCS-CAIN: NIR Survey of the Galactic plane
ivo://cds.vizier/ii/281,II/281,2MASS 6X Point Source Working Database / Catalog
ivo://cds.vizier/ii/293,II/293,GLIMPSE Source Catalog (I + II + 3D)
ivo://cds.vizier/ii/295,II/295,Spitzer IRAC survey of the galactic center
